In [1]:
from pycalrissian.context import CalrissianContext
from pycalrissian.job import CalrissianJob
from pycalrissian.execution import CalrissianExecution
import base64
import os
import yaml
from kubernetes.client.models.v1_job import V1Job

Please make sure you save the gitlab username and password in an environment variables as below:

>```python
>nano ~/.bashrc
># in the end of the file please add your environment variables
>## gitlab creadentials:
>export P_GITLAB_USER='<git userneame>';
>export P_GITLAB_PASS='<git access token>';
># save the file and then restart your terminal as below:
>source ~/.bashrc
>```

make sure you restart the vscode.

In [2]:
username = os.environ.get("P_GITLAB_USER")
password = os.environ.get("P_GITLAB_PASS")
username

'pmembari'

In [3]:



auth = base64.b64encode(f"{username}:{password}".encode("utf-8")).decode(
    "utf-8"
)

secret_config = {
    "auths": {
        "registry.gitlab.com": {
            "auth": auth
        },
    }
}

In [4]:
secret_config

{'auths': {'registry.gitlab.com': {'auth': 'cG1lbWJhcmk6Z2xwYXQtanRaMXZXekpicm5zbnBQd0VGbU4='}}}

In [5]:
namespace_name = "test"

session = CalrissianContext(
            namespace=namespace_name,
            storage_class="nfs",
            volume_size="10G",
            image_pull_secrets=secret_config,
)

In [6]:
session.initialise()

2024-08-26 14:28:49.540 | INFO     | pycalrissian.context:initialise:90 - create role pod-manager-role


2024-08-26 14:28:49.548 | INFO     | pycalrissian.context:initialise:99 - create role binding for role pod-manager-role
2024-08-26 14:28:49.553 | INFO     | pycalrissian.context:initialise:90 - create role log-reader-role
2024-08-26 14:28:49.560 | INFO     | pycalrissian.context:initialise:99 - create role binding for role log-reader-role
2024-08-26 14:28:49.566 | INFO     | pycalrissian.context:initialise:104 - create persistent volume claim 'calrissian-wdir' of 10G with storage class nfs
2024-08-26 14:28:54.587 | INFO     | pycalrissian.context:create_pvc:483 - pvc calrissian-wdir created
2024-08-26 14:28:54.589 | INFO     | pycalrissian.context:initialise:118 - create secret container-rg
2024-08-26 14:28:54.600 | INFO     | pycalrissian.context:initialise:121 - patch service account


In [7]:
with open("./test.cwl", "r") as stream:
    cwl = yaml.safe_load(stream)


with open("./params.yaml", "r") as file:
    params = yaml.safe_load(file)
params

{'input_name': 'Parham'}

In [8]:
os.environ["CALRISSIAN_IMAGE"] = "docker.io/terradue/calrissian:0.12.0"
job = CalrissianJob(
    cwl=cwl,
    params=params,
    runtime_context=session,
    cwl_entry_point="hello-world",
    max_cores=2,
    max_ram="4G",
    tool_logs=True,
)

2024-08-26 14:29:15.624 | INFO     | pycalrissian.job:__init__:69 - using default security context {'runAsUser': 0, 'runAsGroup': 0, 'fsGroup': 0}
2024-08-26 14:29:15.625 | INFO     | pycalrissian.job:__init__:80 - job name: job-1724675355624999-65ea8b46-ed87-4232-a9cf-a4a24be5576d
2024-08-26 14:29:15.625 | INFO     | pycalrissian.job:__init__:81 - create CWL config map
2024-08-26 14:29:20.655 | INFO     | pycalrissian.context:create_configmap:534 - config map cwl-workflow created
2024-08-26 14:29:20.656 | INFO     | pycalrissian.job:__init__:83 - create processing parameters config map
2024-08-26 14:29:25.679 | INFO     | pycalrissian.context:create_configmap:534 - config map params created


In [9]:
isinstance(job.to_k8s_job(), V1Job)

2024-08-26 14:29:25.687 | INFO     | pycalrissian.job:_get_calrissian_container:420 - using Calrissian image: docker.io/terradue/calrissian:0.12.0


True

In [10]:
job.to_yaml("job.yml")

2024-08-26 14:29:25.700 | INFO     | pycalrissian.job:_get_calrissian_container:420 - using Calrissian image: docker.io/terradue/calrissian:0.12.0
2024-08-26 14:29:25.707 | INFO     | pycalrissian.job:to_yaml:144 - job job-1724675355624999-65ea8b46-ed87-4232-a9cf-a4a24be5576d serialized to job.yml


In [11]:
execution = CalrissianExecution(job=job, runtime_context=session)

In [12]:
execution.submit()

2024-08-26 14:29:27.558 | INFO     | pycalrissian.execution:submit:32 - submit job job-1724675355624999-65ea8b46-ed87-4232-a9cf-a4a24be5576d
2024-08-26 14:29:27.560 | INFO     | pycalrissian.job:_get_calrissian_container:420 - using Calrissian image: docker.io/terradue/calrissian:0.12.0
2024-08-26 14:29:27.574 | INFO     | pycalrissian.execution:submit:38 - job job-1724675355624999-65ea8b46-ed87-4232-a9cf-a4a24be5576d submitted


In [19]:
execution.monitor(interval=10)

2024-08-21 17:31:26.539 | INFO     | pycalrissian.execution:monitor:210 - job job-1724254249216659-0850bafd-be15-4cd7-8f10-f4583075deb6 is active
2024-08-21 17:31:36.579 | INFO     | pycalrissian.execution:monitor:210 - job job-1724254249216659-0850bafd-be15-4cd7-8f10-f4583075deb6 is active
2024-08-21 17:31:46.595 | INFO     | pycalrissian.execution:monitor:210 - job job-1724254249216659-0850bafd-be15-4cd7-8f10-f4583075deb6 is active
2024-08-21 17:31:56.610 | INFO     | pycalrissian.execution:monitor:210 - job job-1724254249216659-0850bafd-be15-4cd7-8f10-f4583075deb6 is active


KeyboardInterrupt: 

In [ ]:
log = execution.get_log()
print(log)